<a href="https://colab.research.google.com/github/OfTheByteQuest/Data-Analytics-Learning-and-Projects/blob/main/Chapter-4/Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/OfTheByteQuest/Data-Analytics-Learning-and-Projects.git

%cd Data-Analytics-Learning-and-Projects

!git config --global user.email "harshrathore9910@gmail.com"
!git config --global user.name "OfTheByteQuest"

Cloning into 'Data-Analytics-Learning-and-Projects'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 125 (delta 22), reused 54 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 3.44 MiB | 3.98 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/Data-Analytics-Learning-and-Projects


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Aggrgating Pandas DataFrames

In [3]:
# Importing mandatory lib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Querying and Merging DataFrames

In [3]:
weather =  pd.read_csv("/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/nyc_weather_2018.csv")
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


In [4]:
snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head(), snow_data.shape

(                    date datatype            station attributes  value
 114  2018-01-01T00:00:00     SNOW  GHCND:US1NYWC0019       ,,N,   25.0
 699  2018-01-04T00:00:00     SNOW  GHCND:US1NJBG0015       ,,N,  229.0
 702  2018-01-04T00:00:00     SNOW  GHCND:US1NJBG0017       ,,N,   10.0
 706  2018-01-04T00:00:00     SNOW  GHCND:US1NJBG0018       ,,N,   46.0
 713  2018-01-04T00:00:00     SNOW  GHCND:US1NJES0018       ,,N,   10.0,
 (639, 5))

In [9]:
station_info = pd.read_csv("/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/weather_stations.csv")
station_info.head(), station_info.shape

(                  id                           name   latitude  longitude  \
 0  GHCND:US1CTFR0022        STAMFORD 2.6 SSW, CT US  41.064100 -73.577000   
 1  GHCND:US1CTFR0039          STAMFORD 4.2 S, CT US  41.037788 -73.568176   
 2  GHCND:US1NJBG0001      BERGENFIELD 0.3 SW, NJ US  40.921298 -74.001983   
 3  GHCND:US1NJBG0002  SADDLE BROOK TWP 0.6 E, NJ US  40.902694 -74.083358   
 4  GHCND:US1NJBG0003           TENAFLY 1.3 W, NJ US  40.914670 -73.977500   
 
    elevation  
 0       36.6  
 1        6.4  
 2       20.1  
 3       16.8  
 4       21.6  ,
 (279, 5))

In [6]:
station_info.id.describe()

,id
count,279
unique,279
top,GHCND:USW00094789
freq,1


In [7]:
weather.station.describe()

,station
count,78780
unique,110
top,GHCND:USW00094789
freq,4270


In [10]:
def get_row_count(*dfs):
  return [df.shape[0] for df in dfs]

In [11]:
get_row_count(station_info, weather)

[279, 78780]

In [12]:
def get_info(attr: str, *dfs):
  return list(map(lambda df: getattr(df, attr), dfs))

In [13]:
get_info('shape', station_info, weather)

[(279, 5), (78780, 5)]

#### Merge or Join in Pandas

The `merge()` method in pandas allows you to combine two DataFrames based on one or more common columns or indices, similar to SQL-style joins. This function is essential for integrating datasets and aligning data for analysis.

**Key Parameters of `merge()`:**

- **`right`**: The DataFrame to merge with the caller.
- **`how`**: Specifies the type of merge to perform:
  - `'left'`: All rows from the left DataFrame and matching rows from the right.
  - `'right'`: All rows from the right DataFrame and matching rows from the left.
  - `'outer'`: All rows from both DataFrames, with `NaN` where there are no matches.
  - `'inner'`: Only rows with matching keys in both DataFrames.
  - `'cross'`: Cartesian product of both DataFrames.
- **`on`**: Column or index level names to join on. If not specified, and other join keys are not provided, it defaults to the intersection of the columns in both DataFrames.
- **`left_on`** and **`right_on`**: Specific columns or index levels from the left and right DataFrames to join on.
- **`left_index`** and **`right_index`**: If `True`, use the index (row labels) from the left or right DataFrame as the join key(s).
- **`suffixes`**: A tuple of string suffixes to apply to overlapping column names in the left and right side, respectively.

**Examples:**

1. **Inner Join on a Common Column:**

   ```python
   import pandas as pd

   df1 = pd.DataFrame({
       'ID': [1, 2, 3],
       'Name': ['Alice', 'Bob', 'Charlie']
   })

   df2 = pd.DataFrame({
       'ID': [1, 2, 4],
       'Age': [24, 27, 22]
   })

   merged_df = pd.merge(df1, df2, on='ID', how='inner')
   print(merged_df)
   ```


   **Output:**

   ```
      ID    Name  Age
   0   1   Alice   24
   1   2     Bob   27
   ```


   This merges the two DataFrames on the 'ID' column, including only rows with matching 'ID' values.

2. **Left Join with Different Column Names:**

   ```python
   df1 = pd.DataFrame({
       'EmployeeID': [1, 2, 3],
       'Name': ['Alice', 'Bob', 'Charlie']
   })

   df2 = pd.DataFrame({
       'ID': [1, 2, 4],
       'Age': [24, 27, 22]
   })

   merged_df = pd.merge(df1, df2, left_on='EmployeeID', right_on='ID', how='left')
   print(merged_df)
   ```


   **Output:**

   ```
      EmployeeID     Name   ID   Age
   0           1    Alice  1.0  24.0
   1           2      Bob  2.0  27.0
   2           3  Charlie  NaN   NaN
   ```


   Here, the merge is performed where 'EmployeeID' from `df1` matches 'ID' from `df2`. Rows in `df1` without a corresponding 'ID' in `df2` have `NaN` for 'ID' and 'Age'.

3. **Outer Join:**

   ```python
   df1 = pd.DataFrame({
       'ID': [1, 2, 3],
       'Name': ['Alice', 'Bob', 'Charlie']
   })

   df2 = pd.DataFrame({
       'ID': [2, 3, 4],
       'Age': [30, 25, 22]
   })

   merged_df = pd.merge(df1, df2, on='ID', how='outer', suffixes=('_left', '_right'))
   print(merged_df)
   ```


   **Output:**

   ```
      ID     Name  Age
   0   1    Alice  NaN
   1   2      Bob  30.0
   2   3  Charlie  25.0
   3   4      NaN  22.0
   ```


   This outer join includes all rows from both DataFrames, with `NaN` where there are no matches.

**Considerations:**

- When merging on columns with the same name, specify the `on` parameter. If the column names differ, use `left_on` and `right_on`.
- Use the `suffixes` parameter to distinguish overlapping column names from the left and right DataFrames.
- Be mindful of the `how` parameter to control the type of join, as it determines which rows are included in the resulting DataFrame.

For more detailed information, refer to the [pandas documentation on merging](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).

***Explanation:*** Since, from previous outputs we know that there are in total 279 unique stations for which we have data in *stations* dataframe for which we have data, but, in *weather* dataframe only, 110 of them appears so, by performing the inner join on them, we will get a dataframe in which only info of all the stations for which we have info will be present along with other info in the *weather* dataframe.

In [14]:
# Inner_join dataframe contains all the rows which have common values of
# station in both the dataframes. Since, weather dataframe has multiple values
# the values of columns in the station dataframe, will be repeated for the
# rows where the 'key' values matches.


inner_join = weather.merge(
    station_info, left_on = 'station', right_on = 'id', how='inner'
)
inner_join.sample(5, random_state=0), inner_join.shape

(                      date datatype            station attributes  value  \
 10739  2018-02-17T00:00:00     PRCP  GHCND:USC00066655   ,,7,0700    4.1   
 45188  2018-07-27T00:00:00     SNOW  GHCND:US1NJES0019       ,,N,    0.0   
 59823  2018-10-05T00:00:00     PRCP  GHCND:US1NJES0024       ,,N,    0.0   
 10852  2018-02-17T00:00:00     TMIN  GHCND:USW00094789   ,,W,2400   -2.1   
 46755  2018-08-03T00:00:00     AWND  GHCND:USW00094745       ,,W,    1.8   
 
                       id                              name   latitude  \
 10739  GHCND:USC00066655                PUTNAM LAKE, CT US  41.082500   
 45188  GHCND:US1NJES0019   WEST CALDWELL TWP 1.3 NE, NJ US  40.861500   
 59823  GHCND:US1NJES0024      CEDAR GROVE TWP 0.4 W, NJ US  40.855695   
 10852  GHCND:USW00094789  JFK INTERNATIONAL AIRPORT, NY US  40.639150   
 46755  GHCND:USW00094745     WESTCHESTER CO AIRPORT, NY US  41.062360   
 
        longitude  elevation  
 10739 -73.638600       91.4  
 45188 -74.277500       81.4

In [15]:
# To produce the same result as the last one.
weather.merge(
    station_info.rename({'id':'station'}, axis=1),
    on='station',
    how='inner'
).sample(
    5, random_state=0
)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
10739,2018-02-17T00:00:00,PRCP,GHCND:USC00066655,",,7,0700",4.1,"PUTNAM LAKE, CT US",41.082500,-73.638600,91.4
45188,2018-07-27T00:00:00,SNOW,GHCND:US1NJES0019,",,N,",0.0,"WEST CALDWELL TWP 1.3 NE, NJ US",40.861500,-74.277500,81.4
59823,2018-10-05T00:00:00,PRCP,GHCND:US1NJES0024,",,N,",0.0,"CEDAR GROVE TWP 0.4 W, NJ US",40.855695,-74.235564,108.5
10852,2018-02-17T00:00:00,TMIN,GHCND:USW00094789,",,W,2400",-2.1,"JFK INTERNATIONAL AIRPORT, NY US",40.639150,-73.764010,3.4
46755,2018-08-03T00:00:00,AWND,GHCND:USW00094745,",,W,",1.8,"WESTCHESTER CO AIRPORT, NY US",41.062360,-73.704630,111.9


In [16]:
left_join = station_info.merge(
    weather,
    left_on = 'id',
    right_on = 'station',
    how = 'left'
)

right_join = weather.merge(
    station_info,
    left_on = 'station',
    right_on = 'id',
    how = 'right'
)

outer_join = weather.merge(
    station_info[station_info.name.str.contains('NY')],
    left_on='station',
    right_on='id',
    how='outer',
    indicator=True
)

***Explanation:*** The difference in counts of rows in inner_join dataframe and left_join, right_join and outer_join dataframes as, inner join only contains rows where values of 'id' and 'station' columns matched whereas, that's not the
case in the other datafrmes. Also, since, left_join and right_join, dataframes contains rows of all the values of 'station' columns but, not of 'id' columns, outer_join dataframe have more rows in it.

In [17]:
get_info("shape", inner_join, left_join, right_join, outer_join)

[(78780, 10), (78949, 10), (78949, 10), (78852, 11)]

In [18]:
pd.concat([outer_join.sample(4, random_state=0),
          outer_join[outer_join.station.isna()].head(2)], axis=0)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
74838,2018-01-23T00:00:00,AWND,GHCND:USW00094789,",,W,",6.3,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",40.639150,-73.764010,3.4,both
19325,2018-03-29T00:00:00,SNOW,GHCND:US1NJPS0012,",,N,",0.0,NaN,NaN,NaN,NaN,NaN,left_only
66828,2018-08-09T00:00:00,WT01,GHCND:USW00094728,",,W,",1.0,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7,both
67039,2018-08-28T00:00:00,WSF2,GHCND:USW00094728,",,W,",5.4,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7,both
14527,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJMS0036,"PARSIPPANY TROY HILLS TWP 2.1 E, NJ US",40.865600,-74.385100,64.3,right_only
14528,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJMS0039,"PARSIPPANY TROY HILLS TWP 1.3 WSW, NJ US",40.853314,-74.446957,94.2,right_only


In [19]:
dirty_data = pd.read_csv(
    filepath_or_buffer = "/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/dirty_data.csv",
    index_col = 'date'
).drop_duplicates().drop(columns='SNWD')

In [32]:
dirty_data.head()
dirty_data.shape
dirty_data.station.value_counts()
dirty_data.index.nunique()

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN


(309, 8)

,count
station,
GHCND:USC00280907,247
?,62


259

In [34]:
valid_station = dirty_data.query(
    'station != "?"'
).copy().drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query(
    'station == "?"'
).copy().drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

In [40]:
resultant_df = valid_station.merge(
    station_with_wesf,
    left_index = True,
    right_index = True,
    how='inner',
    suffixes=('', '_?')
)
resultant_df.query('WESF > 0').head()
resultant_df.shape

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


(50, 10)

In [43]:
# To produce the same result as the last one using `.join() method`
valid_station.join(
    station_with_wesf,
    rsuffix = "_?"
).query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


***Note:*** The `.join()` method always uses index column to join dataframes but, can use any column from the right dataframe if provided as the value of `on` parameter.

#### Set Operations

In [45]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

In [44]:
weather.index.difference(station_info.index)

RangeIndex(start=0, stop=279, step=1)

In [46]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

In [47]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

### DataFrame Operations

In [4]:
!head -n 1 '/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/nyc_weather_2018.csv'
!head -n 1 '/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/fb_2018.csv'

date,datatype,station,attributes,value
date,open,high,low,close,volume


In [20]:
weather = pd.read_csv(
    '/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/nyc_weather_2018.csv',
    parse_dates = ['date']
)

fb = pd.read_csv(
    '/content/Data-Analytics-Learning-and-Projects/Chapter-4/Data/fb_2018.csv',
    index_col='date',
    parse_dates = True
)

#### Airthmatic and Statistics

##### Pandas Airthmatic functions

Pandas provide several method to perform airthmatic operations on `DataFrames` and `Series` objects, like `.sub()` or `div()` methods. These operations can be performed with the use of mathematical operators and with the ability of `numpy` to vecotrize these operations and broadcating, they might produce the same effects with certain differes which being:

###### **1. Handling of Different Indexes (Alignment)**  
- Pandas arithmetic methods like `.add()`, `.sub()`, `.div()`, etc., **automatically align indexes** before performing operations.  
- Arithmetic operators (`+`, `-`, `*`, `/`) do not align indexes, which may lead to mismatched computations or NaN values.  

**Example:**  
```python
s1 = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
s2 = pd.Series([1, 2, 3], index=['b', 'c', 'd'])
print(s1 - s2)  # Automatic index alignment
```
**Output:**  
```
a    NaN
b   18.0
c   28.0
d    NaN
dtype: float64
```
If using NumPy, the operation would ignore indexes and only perform element-wise subtraction.

---

###### **2. Handling Missing Values (`fill_value` Parameter)**  
- Pandas methods provide a `fill_value` parameter to replace NaNs before computation.  
- Arithmetic operators will propagate NaN values, leading to missing results.  

**Example:**  
```python
s1 = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
s2 = pd.Series([1, 2, 3], index=['b', 'c', 'd'])
print(s1.sub(s2, fill_value=0))  # Replaces NaN with 0
```
**Output:**  
```
a   10.0  # s1['a'] - 0 (fill value)
b   18.0  # s1['b'] - s2['b']
c   28.0  # s1['c'] - s2['c']
d   -3.0  # 0 (fill value) - s2['d']
dtype: float64
```
Using `-` directly would result in NaN for unmatched indexes.

---

###### **3. Operations Between Different Data Structures**  
- Using arithmetic operators on DataFrames and Series can sometimes lead to broadcasting issues.  
- Pandas arithmetic methods allow explicit row-wise or column-wise operations using the `axis` parameter.  

**Example (Row-wise Operation with Series):**  
```python
df = pd.DataFrame({'A': [10, 20, 30], 'B': [40, 50, 60]})
s = pd.Series([1, 2, 3])  # No column labels
print(df.sub(s, axis=0))  # Explicit row-wise broadcasting
```
**Output:**  
```
    A   B
0   9  39
1  18  48
2  27  57
```
Arithmetic operators do not explicitly specify how broadcasting should happen.

---

###### **4. Readability and Maintainability**  
- Using `df.sub()`, `df.div()`, etc., makes the **operation’s intent clearer**.  
- It avoids confusion when dealing with different data structures and handling NaNs.  

**Comparison:**  
```python
df1.sub(df2, fill_value=0)  # Clearly handles missing values
df1 - df2  # Less explicit handling of missing values
```

---

###### **Comparison Table**  

| Feature | Arithmetic Operators (`+`, `-`, `*`, `/`) | Pandas Methods (`add()`, `sub()`, `div()`, etc.) |
|---------|--------------------------------|--------------------------------|
| **Index Alignment** | No automatic alignment | Auto-aligned by index |
| **Handling NaN Values** | NaNs propagate | `fill_value` avoids NaNs |
| **Operations on Different Shapes** | Can cause mismatches | Allows explicit broadcasting (`axis` parameter) |
| **Readability & Maintainability** | Less explicit | More explicit intent |

---

#### **When to Use Each Approach**  
- **Arithmetic operators:** When working with fully aligned Pandas objects or NumPy arrays.  
- **Pandas arithmetic methods:** When dealing with missing values, misaligned indexes, or requiring explicit broadcasting.

In [21]:
fb.head(2)

,open,high,low,close,volume
date,,,,,
2018-01-02,177.68,181.58,177.55,181.42,18151903
2018-01-03,181.88,184.78,181.33,184.67,16886563


In [22]:
fb.assign(
    abs_z_score_volume = lambda x: x.volume.sub(
        x.volume.mean()
    ).div(x.volume.std()).abs()
).query('abs_z_score_volume > 3')

,open,high,low,close,volume,abs_z_score_volume
date,,,,,,
2018-03-19,177.01,177.17,170.06,172.56,88140060,3.145078
2018-03-20,167.47,170.20,161.95,168.15,129851768,5.315169
2018-03-21,164.80,173.40,163.30,169.39,106598834,4.105413
2018-03-26,160.82,161.10,149.02,160.06,126116634,5.120845
2018-07-26,174.89,180.13,173.75,176.26,169803668,7.393705


In [23]:
# To produce the same response as the lat one

fb.assign(
    abs_z_score_volume = lambda x: ((x.volume - x.volume.mean())/x.volume.std()).abs()
).query('abs_z_score_volume > 3')

,open,high,low,close,volume,abs_z_score_volume
date,,,,,,
2018-03-19,177.01,177.17,170.06,172.56,88140060,3.145078
2018-03-20,167.47,170.20,161.95,168.15,129851768,5.315169
2018-03-21,164.80,173.40,163.30,169.39,106598834,4.105413
2018-03-26,160.82,161.10,149.02,160.06,126116634,5.120845
2018-07-26,174.89,180.13,173.75,176.26,169803668,7.393705


##### **Axes in Pandas and NumPy**

###### **1. Introduction to Axes**
In both **Pandas** and **NumPy**, the `axis` parameter is a fundamental concept that dictates how operations are applied across data structures. It determines whether an operation is performed along rows or columns in **DataFrames**, **Series**, and **NumPy arrays**.

The concept of axes is essential for functions such as **sum(), mean(), apply(), drop(), concatenate(), and many more**.

---

###### **2. Understanding `axis=0` vs. `axis=1`**

**In Pandas (DataFrame and Series)**

| `axis` Value | Direction | Operation Applies To | Examples |
|-------------|----------|---------------------|----------|
| `axis=0` | **Vertical** | Operates on columns (affects rows) | Sum of each column, dropping rows, applying functions column-wise |
| `axis=1` | **Horizontal** | Operates on rows (affects columns) | Sum of each row, dropping columns, applying functions row-wise |

 **In NumPy (Arrays and Matrices)**

| `axis` Value | Direction | Operation Applies To | Examples |
|-------------|----------|---------------------|----------|
| `axis=0` | **Downward** | Column-wise operations | Summing along columns, finding max along columns |
| `axis=1` | **Across** | Row-wise operations | Summing along rows, finding max along rows |

---

###### **3. Axis in Pandas**

**(a) Using `axis` with the `drop()` method**

```python
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

# Drop row at index 1
df_dropped_row = df.drop(1, axis=0)

# Drop column 'B'
df_dropped_column = df.drop('B', axis=1)
```

**(b) Using `axis` with aggregation functions (e.g., `sum()`)**

```python
# Sum along columns (column-wise sum)
df_sum_col = df.sum(axis=0)

# Sum along rows (row-wise sum)
df_sum_row = df.sum(axis=1)
```

**(c) Using `axis` with `apply()` for custom functions**

```python
# Find max in each column
df_max_col = df.apply(max, axis=0)

# Find max in each row
df_max_row = df.apply(max, axis=1)
```

**(d) Using `axis` in element-wise operations**

```python
s = pd.Series([10, 20, 30])

# Subtract Series from DataFrame row-wise (index alignment)
df_subtract_row = df.sub(s, axis=0)

# Subtract Series from DataFrame column-wise (broadcasting)
df_subtract_col = df.sub(s, axis=1)
```

---

###### **4. Axis in NumPy**

**(a) Using `axis` with NumPy Arrays**

```python
import numpy as np

arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# Sum along columns (downward)
sum_col = np.sum(arr, axis=0)

# Sum along rows (across)
sum_row = np.sum(arr, axis=1)
```

**(b) Using `axis` with NumPy's `mean()` and `max()`**

```python
mean_col = np.mean(arr, axis=0)  # Mean of each column
max_row = np.max(arr, axis=1)    # Max of each row
```

---

###### **5. Axis in Concatenation and Stacking**

**(a) In Pandas (`concat()` method)**

```python
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})

# Concatenating along rows (adding rows, column names remain the same)
df_concat_row = pd.concat([df1, df2], axis=0)

# Concatenating along columns (adding new columns)
df_concat_col = pd.concat([df1, df2], axis=1)
```

**(b) In NumPy (`concatenate()` function)**

```python
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[5, 6], [7, 8]])

# Stack arrays vertically (rows added)
arr_stack_vert = np.concatenate((arr1, arr2), axis=0)

# Stack arrays horizontally (columns added)
arr_stack_horiz = np.concatenate((arr1, arr2), axis=1)
```

---

###### **6. Summary and Best Practices**

| Use Case | Pandas | NumPy |
|----------|--------|--------|
| **Summing values** | `df.sum(axis=0)` | `np.sum(arr, axis=0)` |
| **Dropping rows/columns** | `df.drop(..., axis=0/1)` | Not applicable |
| **Applying functions** | `df.apply(func, axis=0/1)` | `np.apply_along_axis(func, axis, arr)` |
| **Arithmetic operations** | `df.sub(s, axis=0/1)` | Element-wise broadcasting |
| **Concatenation** | `pd.concat([df1, df2], axis=0/1)` | `np.concatenate((arr1, arr2), axis=0/1)` |

###### **Best Practices:**
- Use `axis=0` when you want to **apply operations on each column (affecting rows)**.
- Use `axis=1` when you want to **apply operations on each row (affecting columns)**.
- Always check **data alignment** when performing operations across different structures.

Understanding axes helps in optimizing operations and writing efficient Pandas and NumPy code.



##### Understanding the Rank Concept in Pandas

###### Basic Definition
In pandas, the `rank()` function assigns a rank to each value in a Series or DataFrame column based on the value's relative position when arranged in order (ascending by default). Ranking works on the values themselves, not on the indexes.

###### Example Explained
Consider this example:

```python
import pandas as pd
s = pd.Series([3, 1, 4, 1, 5, 9, 2])
s.sort_values()
print(s.rank())  # Default ranking
```

This produces:
```
0    4.0
1    1.5
2    5.0
3    1.5
4    6.0
5    7.0
6    3.0
dtype: float64
```

###### Step-by-Step Explanation

1. Original Series: `[3, 1, 4, 1, 5, 9, 2]` with indexes `[0, 1, 2, 3, 4, 5, 6]`

2. When sorted: `[1, 1, 2, 3, 4, 5, 9]`

3. Rank assignment:
   - Value 3 (at index 0) → 4th smallest value → rank 4.0
   - Value 1 (at index 1) → tied for 1st/2nd smallest → average rank 1.5
   - Value 4 (at index 2) → 5th smallest value → rank 5.0
   - Value 1 (at index 3) → tied for 1st/2nd smallest → average rank 1.5
   - Value 5 (at index 4) → 6th smallest value → rank 6.0
   - Value 9 (at index 5) → 7th smallest value → rank 7.0
   - Value 2 (at index 6) → 3rd smallest value → rank 3.0

4. The output shows each original index with the rank of the value that was at that position.

###### Key Features of Rank()

1. **Tie Handling Methods**:
   - `'average'` (default): Assigns the average of the ranks to tied values
   - `'min'`: Assigns the lowest rank to all tied values
   - `'max'`: Assigns the highest rank to all tied values
   - `'first'`: Ranks assigned based on the order the values appear
   - `'dense'`: Like 'min' but with no gaps between ranks

2. **Float Output**: Ranks are returned as floats to accommodate tie-breaking strategies like 'average'

3. **Direction**: Can rank in ascending (`ascending=True`, default) or descending order (`ascending=False`)

###### Practical Applications

1. **Non-parametric Statistics**: For tests that don't assume normal distribution (Spearman correlation, Mann-Whitney, etc.)

2. **Outlier Resistance**: Rankings are immune to extreme values that might skew analyses

3. **Normalization**: Puts different datasets on a comparable scale

4. **Percentiles and Quantiles**: Foundation for understanding relative positioning of values

5. **Competition Scoring**: Many contests use ranks rather than absolute scores

###### Real-World Example

When comparing metrics across different scales (e.g., salaries across countries with different currencies), ranking can provide a meaningful way to understand relative standing regardless of the original units.

###### Common Pattern in Output

Remember that in the output:
- Left side: Original position (index) in the Series
- Right side: Rank assigned to the value that was at that position

This format sometimes creates confusion since it appears as though the ranks are assigned to indexes, when they're actually computed based on the values. But, pandas may be using this format of giving the output as, <mark>the index values will align with the original index of the dataframe when they are being added as a new column for analyses.</mark>

***Explanation:*** It can be observed that the largest percentage change in volume traded was on January 12, 2018, which was the day, Meta announced that changes to the news feed section, which was to prioritize content from user's friends over brands they follow. Since, most of the Meta's revenue comes from advertising, approximately 99% (as of Dec 31, 2023), this caused panic resulting people to sell it's stocks driving the volume traded to the highest till, Decenmber 31, 2018.

In [24]:
fb.assign(
    volume_pct_change = fb.volume.pct_change(),
    pct_change_rank = lambda x: x.volume_pct_change.abs().rank(ascending=False)
).nsmallest(5, 'pct_change_rank')

,open,high,low,close,volume,volume_pct_change,pct_change_rank
date,,,,,,,
2018-01-12,178.06,181.48,177.40,179.37,77551299,7.087876,1.0
2018-03-19,177.01,177.17,170.06,172.56,88140060,2.611789,2.0
2018-07-26,174.89,180.13,173.75,176.26,169803668,1.628841,3.0
2018-09-21,166.64,167.25,162.81,162.93,45994800,1.428956,4.0
2018-03-26,160.82,161.10,149.02,160.06,126116634,1.352496,5.0


In [25]:
# pd.Series.rank

In [26]:
fb['2018-01-11':'2018-01-12']

,open,high,low,close,volume
date,,,,,
2018-01-11,188.40,188.40,187.38,187.77,9588587
2018-01-12,178.06,181.48,177.40,179.37,77551299


##### Difference between `.any()` and `.all()` method

**1. `.any()`**

`.any()` method looks in the mask, whether atleast one of the values, along the methioned axis, has `True` value. So, until none of the values along the mentioned axis has `True` value, it will give us `True`. Only if, all the values are `False`, along the axis, it will give us `False`. For, `axis = 0` (`default`), it will check divisions along the horizontal axis, which are columns, and give us the results for each column individually whereas, for `axis=1`, the checking happens along the vertical axis, and thus all the rows are examined individually.

**2. `.all()`**

`.all()` methods, in-opposite to `.any()` method, check for the presence of `True` for all the cells, along the mentioned axis and returns `True` only if, all of the cells have `True` value. Functionality, through, `axis` parameter is manipulated in the same manner as it is done in, `.any()` method.

In [36]:
# To check whether any column has value less than 215
(fb > 215).any()
(fb > 215).all()

,0
open,True
high,True
low,False
close,True
volume,True


,0
open,False
high,False
low,False
close,False
volume,True


In [38]:
# pd.Series.any
# pd.Series.all